<a href="https://colab.research.google.com/github/billibilbi/almost_legal_links/blob/main/Copy_of_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer



dataset = load_dataset("csv", data_files="/content/ccass_rapprochements_sommaires_with_sources.csv")


Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
# Create train and validation splits
dataset = dataset['train'].train_test_split(test_size=0.2)  # Split the 'train' split
dataset['validation'] = dataset.pop('test')  # Rename 'test' to 'validation'

In [10]:
# Previous cells should define or load 'dataset'

# Assuming 'dataset' is a Hugging Face DatasetDict
# Access the 'train' split within the DatasetDict
train_dataset = dataset['train']

# Perform the split on the 'train' dataset
tokenized_datasets = train_dataset.train_test_split(test_size=0.2)

# Rename 'test' to 'validation' within the resulting dictionary
tokenized_datasets['validation'] = tokenized_datasets.pop('test')

# Now you can access and modify 'tokenized_datasets'
# ... (rest of your code)

In [11]:
tokenized_datasets = tokenized_datasets['train'].train_test_split(test_size=0.2)
tokenized_datasets['validation'] = tokenized_datasets.pop('test')

In [12]:
from transformers import AutoTokenizer

model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [40]:
def tokenize_function(examples):
    return tokenizer(
        examples['SOMMAIRE_SOURCE'],
        examples['JUDILIBRE_SOURCE'],
        examples['SOMMAIRE_RAPPROCHEMENT'],
        examples['JUDILIBRE_RAPPROCHEMENT'],
        padding='max_length',
        truncation=True
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)



In [14]:
print(tokenized_datasets)
print (dataset)

DatasetDict({
    train: Dataset({
        features: ['SOMMAIRE_SOURCE', 'JUDILIBRE_SOURCE', 'SOMMAIRE_RAPPROCHEMENT', 'JUDILIBRE_RAPPROCHEMENT', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 6272
    })
    validation: Dataset({
        features: ['SOMMAIRE_SOURCE', 'JUDILIBRE_SOURCE', 'SOMMAIRE_RAPPROCHEMENT', 'JUDILIBRE_RAPPROCHEMENT', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1568
    })
})
DatasetDict({
    train: Dataset({
        features: ['SOMMAIRE_SOURCE', 'JUDILIBRE_SOURCE', 'SOMMAIRE_RAPPROCHEMENT', 'JUDILIBRE_RAPPROCHEMENT'],
        num_rows: 6272
    })
    validation: Dataset({
        features: ['SOMMAIRE_SOURCE', 'JUDILIBRE_SOURCE', 'SOMMAIRE_RAPPROCHEMENT', 'JUDILIBRE_RAPPROCHEMENT'],
        num_rows: 1568
    })
})


In [15]:
from transformers import AutoModelForSequenceClassification

# Number of unique labels in your dataset
num_labels = len(dataset['train'].unique('SOMMAIRE_RAPPROCHEMENT'))

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

Flattening the indices:   0%|          | 0/6272 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AFTER THIS


In [20]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=6,
    weight_decay=0.01,
)

In [38]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

The error message "ValueError: Expected input batch_size (8) to match target batch_size (4096)" indicates a mismatch between the batch size of your input data and the batch size of your target labels during training.

This is likely happening because the tokenize_function is tokenizing four separate text columns and concatenating them into a single input sequence. This creates an input with a batch size of 8 (as defined in TrainingArguments). However, the target labels ('SOMMAIRE_RAPPROCHEMENT') still have a batch size corresponding to the original dataset, which is 4096 in this case. The model expects the input and target batch sizes to match, leading to the ValueError.

In [43]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,  # Define your custom metrics function
    tokenizer=tokenizer,
)

NameError: name 'data_collator' is not defined

In [36]:
trainer.train()

ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.